In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import numpy as np

from PIL import Image, ImageOps
from collections import defaultdict
import sys
sys.path.insert(0, '/media/allen/mass/deep-learning-works/')
from data import data_manager
from config.config_manager import _C as cfg, build_output
from tools.logger import setup_logger

logger = setup_logger(".")
cfg.merge_from_file("../reid_trick.yml")
cfg.DATASET.TRAIN_PATH = "/media/allen/mass/reid/"
cfg.DATASET.NAME = 'market1501'

In [ ]:
from data.build_data import build_reid_dataset
dataset = data_manager.init_img_dataset(cfg)
pil_gallery = build_reid_dataset(dataset.gallery, return_indice=True)
pil_query = build_reid_dataset(dataset.query, return_indice=True)

In [ ]:
rs = np.load("evaluation/reid/OSNet_merge_cels_triplet_center_Adam_lr_0.00035_warmup_10_0.01_plateau_10_flip_crop_erase_imagenet/2019-11-07_16-46-36/rs.npy")
fg_ = np.load("evaluation/reid/OSNet_merge_cels_triplet_center_Adam_lr_0.00035_warmup_10_0.01_plateau_10_flip_crop_erase_imagenet/2019-11-07_16-46-36/last.npy")

In [ ]:
fg = {}
for i in range(rs.shape[0]):
    remove = fg_[i] == -1
    fg[i] = fg_[i][~remove]

In [ ]:
hard_query = np.arange(len(rs))[rs >= 10000]

hard_gallery = last[rs >= 10000]

def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

os.makedirs('/home/allen.jc.lin/market1501/hard_sample')

for i, (q, g) in enumerate(zip(hard_query, hard_gallery)):
    img = get_concat_h(pil_query[q][0], pil_gallery[g][0])
    img.save('/home/allen.jc.lin/market1501/hard_sample/{}.jpg'.format(i))

In [ ]:
for i in range(rs.shape[0]):
    img, q_pid, _, _ = pil_query[i]
    img = ImageOps.expand(img,border=5,fill='yellow')
    for j in range(20):
        g_img, g_pid, _, _ = pil_gallery[fg[i][j]]
        if g_pid != q_pid:
            g_img = ImageOps.expand(g_img,border=5,fill='red')
        else:
            g_img = ImageOps.expand(g_img,border=5,fill='green')
        img = get_concat_h(img, g_img)
    break